In [1]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.10 ; cuda:  1.10.0


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("road_defect_detection_train", {}, "instances_default.json", "training_images")
register_coco_instances("road_defect_detection_val", {}, "new_instances.json", "validation_images")

my_dataset_train_metadata = MetadataCatalog.get("road_defect_detection_train")
dataset_dicts = DatasetCatalog.get("road_defect_detection_train")

[05/28 08:23:20 d2.data.datasets.coco]: Loaded 269 images in COCO format from instances_default.json


In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.WEIGHTS = 'model_final.pth'
cfg.DATASETS.TRAIN = ("road_defect_detection_train",)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [7]:
from detectron2.utils.visualizer import ColorMode
# import matplotlib.pyplot as plt
test_path = "validation_images"

for filename in os.listdir(test_path):
    f = os.path.join(test_path, filename)
    print(f)
    im = cv2.imread(f)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out_filename = filename[:-4] + "_out.jpg"
    cv2.imwrite(out_filename, v.get_image()[:, :, ::-1])

validation_images\cropped_1.png
validation_images\cropped_10.png
validation_images\cropped_11.png
validation_images\cropped_12.png
validation_images\cropped_13.png
validation_images\cropped_14.png
validation_images\cropped_15.png
validation_images\cropped_16.png
validation_images\cropped_17.png
validation_images\cropped_18.png
validation_images\cropped_19.png
validation_images\cropped_2.png
validation_images\cropped_20.png
validation_images\cropped_21.png
validation_images\cropped_22.png
validation_images\cropped_23.png
validation_images\cropped_24.png
validation_images\cropped_25.png
validation_images\cropped_26.png
validation_images\cropped_27.png
validation_images\cropped_28.png
validation_images\cropped_29.png
validation_images\cropped_3.png
validation_images\cropped_30.png
validation_images\cropped_31.png
validation_images\cropped_32.png
validation_images\cropped_33.png
validation_images\cropped_34.png
validation_images\cropped_35.png
validation_images\cropped_36.png
validation_im

In [5]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("road_defect_detection_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "road_defect_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[05/28 08:23:32 d2.data.datasets.coco]: Loaded 68 images in COCO format from new_instances.json
[05/28 08:23:32 d2.data.build]: Distribution of instances among all 8 categories:
|  category  | #instances   |   category    | #instances   |   category    | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|   Cracks   | 31           |   Potholes    | 33           |    Patches    | 0            |
|  Sidewalk  | 0            | White Road .. | 0            | Double Yell.. | 0            |
|    Car     | 0            | Alligator C.. | 4            |               |              |
|   total    | 68           |               |              |               |              |
[05/28 08:23:32 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/28 08:23:32 d2.data.common]: Serializing 68 elements to byte tensors and concatenating 

C:\Users\luope\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/28 08:23:50 d2.evaluation.evaluator]: Inference done 1/68. Dataloading: 10.0837 s/iter. Inference: 8.3632 s/iter. Eval: 0.0012 s/iter. Total: 18.4486 s/iter. ETA=0:20:36
[05/28 08:23:55 d2.evaluation.evaluator]: Inference done 24/68. Dataloading: 0.0006 s/iter. Inference: 0.2210 s/iter. Eval: 0.0010 s/iter. Total: 0.2226 s/iter. ETA=0:00:09
[05/28 08:24:00 d2.evaluation.evaluator]: Inference done 47/68. Dataloading: 0.0006 s/iter. Inference: 0.2188 s/iter. Eval: 0.0009 s/iter. Total: 0.2204 s/iter. ETA=0:00:04
[05/28 08:24:05 d2.evaluation.evaluator]: Total inference time: 0:00:13.969117 (0.221732 s / iter per device, on 1 devices)
[05/28 08:24:05 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:13 (0.215731 s / iter per device, on 1 devices)
[05/28 08:24:05 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/28 08:24:05 d2.evaluation.coco_evaluation]: Saving results to ./output\coco_instances_results.json
[05/28 08:24:05 d2.evaluation.coco_ev

In [8]:
print(outputs)

{'instances': Instances(num_instances=2, image_height=276, image_width=583, fields=[pred_boxes: Boxes(tensor([[ 32.3418,  94.9284, 530.5706, 190.1936],
        [ 35.2642,   4.0985,  96.4415,  57.9806]], device='cuda:0')), scores: tensor([0.9740, 0.8684], device='cuda:0'), pred_classes: tensor([1, 0], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, Fals

In [ ]:
test_path = "validation_images"
with open("new_instances.json", "r") as read_file:
    instances = json.load(read_file)

for filename in os.listdir(test_path):
    i = 0
    f = os.path.join(test_path, filename)
    print(f)
    im = cv2.imread(f)
    outputs = instances["annotations"][i]
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out_filename = filename[:-4] + "_out.jpg"
    cv2.imwrite(out_filename, v.get_image()[:, :, ::-1])
    i += 1